In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load_word2vec_format('/home/retkowski/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [2]:
#model.init_sims(replace=True)
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

In [14]:
model.doesnt_match("Are we save".split())

'save'

In [18]:
model.most_similar(positive=['Clinton'])

[('Hillary_Rodham_Clinton', 0.520073652267456),
 ('Hillary_Clinton', 0.5176734328269958),
 ('Obama', 0.5171620845794678),
 ('Illinois_senator', 0.47700589895248413),
 ('McCain', 0.46960338950157166),
 ('Senator_Hillary_Clinton', 0.46645233035087585),
 ('Clintons', 0.46552467346191406),
 ('Sen._Hillary_Clinton', 0.45665228366851807),
 ('Rodham_Clinton', 0.452339231967926),
 ('Barack_Obama', 0.4459362328052521)]

In [29]:
print(model.similarity("chaos","sports"))
print(model.similarity("chaos","politics"))
print(model.similarity("chaos","style"))
print(model.similarity("chaos","economy"))
print(model.similarity("chaos","technology"))

0.0347451738013
0.252810442651
0.153951080801
0.175373157659
0.00543596835089


In [33]:
model.clear_sims()

In [1]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load_word2vec_format('/home/retkowski/glove/glove.6B.100d.txt', binary=False)

ValueError: invalid literal for int() with base 10: 'the'

In [41]:
%xdel lines

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


w2v = None

with open("/home/retkowski/glove/glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [15]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec], axis=0)
            for words in X
        ])

In [3]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.items())))

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from collections import defaultdict

#etree_w2v = Pipeline([
#    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
#    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
tree_w2v_tfidf = Pipeline([
   ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
   ("extra trees", ExtraTreesClassifier(n_estimators=50))])

In [8]:
X = [['Berlin', 'London', 'Madrid'],
     ['cow', 'cat', 'dog'],
     ['pink', 'yellow', 'red']]
y = ['capitals', 'animals', 'colors']
tree_w2v_tfidf.fit(X, y)

# never before seen words!!!
test_X = [['dog'], ['red'], ['Madrid']]

print(tree_w2v_tfidf.predict(test_X))

['animals' 'animals' 'animals']
